# Comprehensive Time Series EDA - VN2 Competition

**Parallel Processing on Apple M2 Max (12 cores, 64GB RAM)**

This notebook performs deep exploratory data analysis on ~3 years of weekly sales data:
- Summary statistics & dispersion coefficients
- Stationarity & heteroskedasticity tests (ADF, KPSS, ARCH)
- ACF/PACF analysis
- Lag analysis & lead-lag relationships
- Frequency domain analysis (periodograms)
- Master SLURP construction with metadata preservation
- Bootstrap analysis leveraging relationship preservation

**Optimization**: All analyses parallelized across 11 workers, memory-optimized data structures.


In [ ]:
# Setup: Environment & Imports
import os
os.environ['OMP_NUM_THREADS'] = '12'
os.environ['MKL_NUM_THREADS'] = '12'
os.environ['OPENBLAS_NUM_THREADS'] = '12'

import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from multiprocessing import cpu_count
import warnings
import time
from contextlib import contextmanager

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

N_JOBS = 11  # Leave 1 core for system
print(f"🚀 Using {N_JOBS} parallel workers on {cpu_count()} cores")
print(f"📊 NumPy using {np.show_config()}")

@contextmanager
def timer(name):
    start = time.perf_counter()
    yield
    elapsed = time.perf_counter() - start
    print(f"⏱️  {name}: {elapsed:.2f}s")


## 1. Load & Prepare Data

Load all raw files and reshape to long format for analysis.


In [ ]:
with timer("Data loading"):
    # Load raw files
    sales = pd.read_csv('../data/raw/Week 0 - 2024-04-08 - Sales.csv')
    stock = pd.read_csv('../data/raw/Week 0 - In Stock.csv')
    master = pd.read_csv('../data/raw/Week 0 - Master.csv')
    
    # Melt to long format
    sales_long = sales.melt(
        id_vars=['Store', 'Product'], 
        var_name='week', 
        value_name='sales'
    )
    sales_long['week_date'] = pd.to_datetime(sales_long['week'])
    sales_long['year'] = sales_long['week_date'].dt.year
    sales_long['retail_week'] = sales_long['week_date'].dt.isocalendar().week
    sales_long['month'] = sales_long['week_date'].dt.month
    
    stock_long = stock.melt(
        id_vars=['Store', 'Product'], 
        var_name='week', 
        value_name='in_stock'
    )
    
    # Merge
    df = sales_long.merge(
        stock_long, 
        on=['Store', 'Product', 'week']
    ).merge(
        master,
        on=['Store', 'Product']
    ).sort_values(['Store', 'Product', 'week']).reset_index(drop=True)

# Memory optimization
df_optimized = df.copy()
df_optimized['Store'] = df_optimized['Store'].astype('int16')
df_optimized['Product'] = df_optimized['Product'].astype('int16')
df_optimized['sales'] = df_optimized['sales'].astype('float32')
df_optimized['year'] = df_optimized['year'].astype('int16')
df_optimized['retail_week'] = df_optimized['retail_week'].astype('int8')
df_optimized['month'] = df_optimized['month'].astype('int8')
df_optimized['in_stock'] = df_optimized['in_stock'].astype('bool')

for col in ['ProductGroup', 'Division', 'Department', 'DepartmentGroup', 'StoreFormat', 'Format']:
    if col in df_optimized.columns:
        df_optimized[col] = df_optimized[col].astype('category')

mem_before = df.memory_usage(deep=True).sum() / 1024**2
mem_after = df_optimized.memory_usage(deep=True).sum() / 1024**2

df = df_optimized  # Use optimized version

print(f"\n📦 Data shape: {df.shape}")
print(f"📅 Date range: {df['week_date'].min()} to {df['week_date'].max()}")
print(f"🏪 Stores: {df['Store'].nunique()}, Products: {df['Product'].nunique()}")
print(f"💾 Memory: {mem_before:.1f} MB → {mem_after:.1f} MB ({(1-mem_after/mem_before)*100:.1f}% reduction)")
print(f"\n✅ All data loaded and in memory")


## 2. Master SLURP Construction

Create one grand SLURP where each row = one SKU-week observation with all metadata preserved for bootstrap analysis.


In [ ]:
from vn2.uncertainty import SLURP

with timer("SLURP construction"):
    # Create scenario ID
    df['scenario_id'] = df.index
    
    # Build SIP dictionary with all variables
    sip_dict = {
        'sales': df['sales'].values,
        'store': df['Store'].values,
        'product': df['Product'].values,
        'retail_week': df['retail_week'].values,
        'year': df['year'].values,
        'month': df['month'].values,
        'in_stock': df['in_stock'].astype(int).values,  # 1/0 for True/False
        'product_group': df['ProductGroup'].cat.codes.values,
        'department': df['Department'].cat.codes.values,
        'store_format': df['StoreFormat'].cat.codes.values,
    }
    
    # Metadata
    meta_df = pd.DataFrame({
        'field_type': ['continuous', 'discrete', 'discrete', 'discrete', 'discrete', 'discrete',
                       'binary', 'discrete', 'discrete', 'discrete'],
        'description': ['Weekly sales', 'Store ID', 'Product ID', 'ISO week', 'Year', 'Month',
                        'Stock availability', 'Product group', 'Department', 'Store format']
    }, index=list(sip_dict.keys())).T
    
    # Create master SLURP
    master_slurp = SLURP.from_dict(
        sip_dict, 
        meta=meta_df,
        provenance="VN2 Historical Demand 2021-2024 with Metadata"
    )
    
    # Save
    master_slurp.to_xml('../data/processed/master_demand_slurp.xml', csvr=4, average=True, median=True)
    df.to_parquet('../data/processed/demand_long.parquet')

print(f"✅ SLURP created: {master_slurp.n_scenarios:,} scenarios")
print(f"   Variables: {list(master_slurp.names)}")
print(f"   Saved to: ../data/processed/master_demand_slurp.xml")


## 3. Summary Statistics & Dispersion (Parallel)

Compute comprehensive statistics for all 599 SKUs in parallel.


In [ ]:
from scipy.stats import skew, kurtosis

def compute_sku_summary(sku_data):
    """Compute all summary stats for one SKU"""
    store, product = sku_data.name
    sales = sku_data['sales'].values
    non_zero = sales[sales > 0]
    
    return {
        'Store': store,
        'Product': product,
        'n_weeks': len(sales),
        'mean': sales.mean(),
        'std': sales.std(),
        'min': sales.min(),
        'q25': np.percentile(sales, 25),
        'median': np.median(sales),
        'q75': np.percentile(sales, 75),
        'max': sales.max(),
        'cv': sales.std() / (sales.mean() + 1e-9),
        'qcd': (np.percentile(sales, 75) - np.percentile(sales, 25)) / 
               (np.percentile(sales, 75) + np.percentile(sales, 25) + 1e-9),
        'mad': np.median(np.abs(sales - np.median(sales))),
        'mad_mean_ratio': np.median(np.abs(sales - np.median(sales))) / (sales.mean() + 1e-9),
        'iqr': np.percentile(sales, 75) - np.percentile(sales, 25),
        'range': sales.max() - sales.min(),
        'skewness': skew(sales),
        'kurtosis': kurtosis(sales),
        'pct_zeros': (sales == 0).mean(),
        'pct_stockout': (~sku_data['in_stock']).mean(),
        'adi': len(sales) / len(non_zero) if len(non_zero) > 0 else np.inf,
    }

with timer("Summary statistics (parallel)"):
    sku_groups = list(df.groupby(['Store', 'Product']))
    summary_list = Parallel(n_jobs=N_JOBS, verbose=1)(
        delayed(compute_sku_summary)(grp) for _, grp in sku_groups
    )
    summary = pd.DataFrame(summary_list)
    summary.to_parquet('../data/processed/summary_statistics.parquet')

print(f"\n✅ Summary stats computed for {len(summary)} SKUs")
print(f"\nOverall statistics:")
print(summary[['mean', 'std', 'cv', 'skewness', 'pct_zeros', 'pct_stockout']].describe())


### 3.1 Visualize Dispersion


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

# CV vs Mean (Heteroskedasticity)
summary.plot.scatter(x='mean', y='cv', alpha=0.3, ax=axes[0], s=10)
axes[0].set_title('CV vs Mean (Heteroskedasticity)')
axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlabel('Mean Sales')
axes[0].set_ylabel('Coefficient of Variation')

# MAD/Mean vs Mean
summary.plot.scatter(x='mean', y='mad_mean_ratio', alpha=0.3, ax=axes[1], s=10, c='orange')
axes[1].set_title('MAD/Mean Ratio vs Mean')
axes[1].set_xscale('log')
axes[1].set_xlabel('Mean Sales')
axes[1].set_ylabel('MAD/Mean Ratio')

# Skewness distribution
summary['skewness'].hist(bins=50, ax=axes[2], edgecolor='black', alpha=0.7)
axes[2].set_title('Skewness Distribution')
axes[2].set_xlabel('Skewness')
axes[2].axvline(0, color='red', linestyle='--', alpha=0.5)

# Kurtosis distribution
summary['kurtosis'].hist(bins=50, ax=axes[3], edgecolor='black', alpha=0.7, color='green')
axes[3].set_title('Excess Kurtosis Distribution')
axes[3].set_xlabel('Excess Kurtosis')
axes[3].axvline(0, color='red', linestyle='--', alpha=0.5)

# Intermittency: Zeros vs ADI
summary.plot.scatter(x='pct_zeros', y='adi', alpha=0.3, ax=axes[4], s=10, c='purple')
axes[4].set_title('Intermittency: % Zeros vs ADI')
axes[4].set_xlabel('% Zeros')
axes[4].set_ylabel('Average Demand Interval')
axes[4].set_yscale('log')

# Stockout vs CV
summary.plot.scatter(x='pct_stockout', y='cv', alpha=0.3, ax=axes[5], s=10, c='red')
axes[5].set_title('Stockout Rate vs CV')
axes[5].set_xlabel('% Stockout Weeks')
axes[5].set_ylabel('Coefficient of Variation')
axes[5].set_yscale('log')

plt.tight_layout()
plt.savefig('../reports/dispersion_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"💾 Saved: ../reports/dispersion_analysis.png")


## 4. Stationarity & Heteroskedasticity Tests (Parallel)

ADF (Augmented Dickey-Fuller), KPSS, and ARCH-LM tests for all SKUs.


In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import het_arch
from scipy import signal

def test_stationarity_parallel(sku_data):
    """Test stationarity & heteroskedasticity for one SKU"""
    store, product = sku_data.name
    series = sku_data['sales'].dropna()
    
    results = {'Store': store, 'Product': product}
    
    # ADF test (null: non-stationary)
    try:
        adf = adfuller(series, autolag='AIC', maxlag=12)
        results['adf_stat'] = adf[0]
        results['adf_pval'] = adf[1]
        results['adf_stationary'] = adf[1] < 0.05
    except:
        results['adf_stat'] = results['adf_pval'] = results['adf_stationary'] = np.nan
    
    # KPSS test (null: stationary)
    try:
        kpss_res = kpss(series, regression='c', nlags='auto')
        results['kpss_stat'] = kpss_res[0]
        results['kpss_pval'] = kpss_res[1]
        results['kpss_stationary'] = kpss_res[1] > 0.05
    except:
        results['kpss_stat'] = results['kpss_pval'] = results['kpss_stationary'] = np.nan
    
    # ARCH test (conditional heteroskedasticity)
    try:
        detrended = signal.detrend(series)
        if len(detrended) > 10:
            arch = het_arch(detrended, nlags=min(5, len(detrended)//4))
            results['arch_stat'] = arch[0]
            results['arch_pval'] = arch[1]
            results['heteroskedastic'] = arch[1] < 0.05
        else:
            results['arch_stat'] = results['arch_pval'] = results['heteroskedastic'] = np.nan
    except:
        results['arch_stat'] = results['arch_pval'] = results['heteroskedastic'] = np.nan
    
    return results

with timer("Stationarity tests (parallel)"):
    stationarity_list = Parallel(n_jobs=N_JOBS, verbose=1)(
        delayed(test_stationarity_parallel)(grp) for _, grp in sku_groups
    )
    stationarity_tests = pd.DataFrame(stationarity_list)
    stationarity_tests.to_parquet('../data/processed/stationarity_tests.parquet')

print(f"\n✅ Stationarity tests complete for {len(stationarity_tests)} SKUs")
print(f"\nTest results:")
print(f"  ADF stationary: {stationarity_tests['adf_stationary'].sum()} / {len(stationarity_tests)} ({stationarity_tests['adf_stationary'].mean()*100:.1f}%)")
print(f"  KPSS stationary: {stationarity_tests['kpss_stationary'].sum()} / {len(stationarity_tests)} ({stationarity_tests['kpss_stationary'].mean()*100:.1f}%)")
print(f"  Heteroskedastic (ARCH): {stationarity_tests['heteroskedastic'].sum()} / {len(stationarity_tests)} ({stationarity_tests['heteroskedastic'].mean()*100:.1f}%)")


## 5. ACF/PACF Analysis (Parallel)

Autocorrelation and partial autocorrelation for lag identification.


In [ ]:
from statsmodels.tsa.stattools import acf, pacf

def compute_acf_pacf_parallel(sku_data, nlags=52):
    """Compute ACF/PACF for one SKU"""
    store, product = sku_data.name
    series = sku_data['sales'].dropna()
    
    if len(series) < nlags + 1:
        return None
    
    try:
        acf_vals = acf(series, nlags=nlags, fft=True)  # FFT faster
        pacf_vals = pacf(series, nlags=nlags, method='ywm')
        
        # Find significant lags
        n = len(series)
        threshold = 1.96 / np.sqrt(n)
        sig_lags_acf = np.where(np.abs(acf_vals[1:]) > threshold)[0] + 1
        sig_lags_pacf = np.where(np.abs(pacf_vals[1:]) > threshold)[0] + 1
        
        return {
            'Store': store,
            'Product': product,
            'acf': acf_vals,
            'pacf': pacf_vals,
            'sig_lags_acf': sig_lags_acf,
            'sig_lags_pacf': sig_lags_pacf,
            'n_sig_acf': len(sig_lags_acf),
            'n_sig_pacf': len(sig_lags_pacf)
        }
    except:
        return None

with timer("ACF/PACF computation (parallel)"):
    acf_list = Parallel(n_jobs=N_JOBS, verbose=1)(
        delayed(compute_acf_pacf_parallel)(grp, nlags=52) for _, grp in sku_groups
    )
    acf_results = {(r['Store'], r['Product']): r for r in acf_list if r is not None}

# Aggregate ACF/PACF across SKUs
all_acf_data = []
for k, v in acf_results.items():
    for lag in range(len(v['acf'])):
        all_acf_data.append({
            'lag': lag,
            'acf': v['acf'][lag],
            'pacf': v['pacf'][lag] if lag < len(v['pacf']) else np.nan
        })

all_acf_df = pd.DataFrame(all_acf_data)
median_acf = all_acf_df.groupby('lag')[['acf', 'pacf']].median()

print(f"✅ ACF/PACF computed for {len(acf_results)} SKUs")

# Plot aggregate ACF/PACF
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].stem(median_acf.index, median_acf['acf'], basefmt=' ')
axes[0].axhline(0, color='black', linewidth=0.8)
axes[0].axhline(1.96/np.sqrt(157), color='red', linestyle='--', label='95% CI')
axes[0].axhline(-1.96/np.sqrt(157), color='red', linestyle='--')
axes[0].set_title('Median ACF Across All SKUs')
axes[0].set_xlabel('Lag (weeks)')
axes[0].set_ylabel('ACF')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].stem(median_acf.index, median_acf['pacf'], basefmt=' ')
axes[1].axhline(0, color='black', linewidth=0.8)
axes[1].axhline(1.96/np.sqrt(157), color='red', linestyle='--', label='95% CI')
axes[1].axhline(-1.96/np.sqrt(157), color='red', linestyle='--')
axes[1].set_title('Median PACF Across All SKUs')
axes[1].set_xlabel('Lag (weeks)')
axes[1].set_ylabel('PACF')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/acf_pacf_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"💾 Saved: ../reports/acf_pacf_analysis.png")


## 6. Frequency Domain Analysis (Vectorized)

Periodogram analysis to identify dominant periodicities.


In [ ]:
from scipy.fft import rfft, rfftfreq

with timer("Frequency analysis (vectorized)"):
    # Reshape to matrix [n_skus, n_weeks]
    sales_pivot = df.pivot(index=['Store', 'Product'], columns='week', values='sales').fillna(0)
    sales_matrix = sales_pivot.values
    
    # Detrend
    sales_centered = sales_matrix - sales_matrix.mean(axis=1, keepdims=True)
    
    # FFT on all SKUs at once
    fft_result = rfft(sales_centered, axis=1)
    power = np.abs(fft_result) ** 2
    
    # Frequencies
    n_weeks = sales_matrix.shape[1]
    freqs = rfftfreq(n_weeks, d=1.0)
    
    # Find top 5 dominant frequencies per SKU
    top_k = 5
    dominant_freq_idx = np.argsort(power, axis=1)[:, -top_k:][:, ::-1]

# Extract dominant periods
dominant_periods_list = []
for i, (store, product) in enumerate(sales_pivot.index):
    idx = dominant_freq_idx[i]
    dominant_freqs = freqs[idx]
    periods = 1.0 / (dominant_freqs + 1e-9)
    periods = periods[(periods > 1) & (periods < 100)]
    
    dominant_periods_list.append({
        'Store': store,
        'Product': product,
        'n_dominant_periods': len(periods),
        'top_period': periods[0] if len(periods) > 0 else np.nan
    })

dominant_periods_df = pd.DataFrame(dominant_periods_list)

print(f"✅ Frequency analysis complete")
print(f"\nDominant period distribution:")
print(dominant_periods_df['top_period'].describe())

# Plot sample periodograms
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

sample_idx = [0, 100, 200, 300]
for ax_i, data_i in enumerate(sample_idx):
    if data_i < len(sales_pivot):
        store, product = sales_pivot.index[data_i]
        psd = power[data_i]
        
        ax = axes[ax_i]
        ax.semilogy(freqs[1:], psd[1:])  # Skip DC component
        ax.set_title(f'Store {store}, Product {product}')
        ax.set_xlabel('Frequency (cycles/week)')
        ax.set_ylabel('Power Spectral Density')
        ax.grid(True, alpha=0.3)
        
        # Mark dominant periods
        idx = dominant_freq_idx[data_i]
        for j in idx[:3]:
            if freqs[j] > 0:
                period = 1.0 / freqs[j]
                if 2 < period < 60:
                    ax.axvline(freqs[j], color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('../reports/frequency_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"💾 Saved: ../reports/frequency_analysis.png")


## 7. SLURP-Based Bootstrap Analysis (Parallel)

Bootstrap with relationship preservation across store, SKU, week, year, and stockout status.


In [ ]:
def bootstrap_chunk(slurp_data, chunk_id, n_bootstrap_per_chunk, seed_offset):
    """Bootstrap a chunk of iterations"""
    np.random.seed(seed_offset + chunk_id)
    
    results = []
    for b in range(n_bootstrap_per_chunk):
        # Sample with replacement (preserves relationships)
        idx = np.random.choice(len(slurp_data), size=len(slurp_data), replace=True)
        sample = slurp_data.iloc[idx]
        
        # Compute aggregates
        overall_mean = sample['sales'].mean()
        overall_std = sample['sales'].std()
        
        # By stockout status
        stockout_mask = sample['in_stock'] == 0
        instock_mask = sample['in_stock'] == 1
        
        stockout_mean = sample[stockout_mask]['sales'].mean() if stockout_mask.any() else np.nan
        instock_mean = sample[instock_mask]['sales'].mean() if instock_mask.any() else np.nan
        
        # By year
        year_means = {}
        for year in sample['year'].unique():
            year_means[f'year_{year}'] = sample[sample['year'] == year]['sales'].mean()
        
        # By retail week (seasonal)
        week_means = {}
        for week in range(1, 53):
            week_data = sample[sample['retail_week'] == week]
            if len(week_data) > 0:
                week_means[f'week_{week}'] = week_data['sales'].mean()
        
        result = {
            'overall_mean': overall_mean,
            'overall_std': overall_std,
            'stockout_mean': stockout_mean,
            'instock_mean': instock_mean,
            **year_means,
            **week_means
        }
        results.append(result)
    
    return results

with timer("Bootstrap analysis (parallel)"):
    n_bootstrap_total = 10000
    n_chunks = N_JOBS
    n_per_chunk = n_bootstrap_total // n_chunks
    
    bootstrap_results = Parallel(n_jobs=N_JOBS, verbose=1)(
        delayed(bootstrap_chunk)(master_slurp.data, i, n_per_chunk, 42) 
        for i in range(n_chunks)
    )
    
    # Flatten results
    bootstrap_flat = [item for chunk in bootstrap_results for item in chunk]
    bootstrap_df = pd.DataFrame(bootstrap_flat)
    
    # Save
    import pickle
    with open('../data/processed/bootstrap_distributions.pkl', 'wb') as f:
        pickle.dump(bootstrap_df, f)

print(f"✅ Bootstrap complete: {len(bootstrap_flat):,} iterations")
print(f"\nOverall demand statistics:")
print(f"  Mean: {bootstrap_df['overall_mean'].mean():.2f} ± {bootstrap_df['overall_mean'].std():.2f}")
print(f"  Std: {bootstrap_df['overall_std'].mean():.2f} ± {bootstrap_df['overall_std'].std():.2f}")
print(f"\nBy stock status:")
print(f"  In-stock mean: {bootstrap_df['instock_mean'].mean():.2f} ± {bootstrap_df['instock_mean'].std():.2f}")
print(f"  Stockout mean: {bootstrap_df['stockout_mean'].mean():.2f} ± {bootstrap_df['stockout_mean'].std():.2f}")


### 7.1 Bootstrap Visualizations


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Overall mean distribution
axes[0, 0].hist(bootstrap_df['overall_mean'], bins=50, density=True, alpha=0.7, edgecolor='black')
axes[0, 0].axvline(bootstrap_df['overall_mean'].mean(), color='red', linestyle='--', label='Mean')
axes[0, 0].set_title('Bootstrap: Overall Mean Demand')
axes[0, 0].set_xlabel('Mean Demand')
axes[0, 0].legend()

# By stockout status
axes[0, 1].hist(bootstrap_df['instock_mean'].dropna(), bins=50, density=True, alpha=0.7, label='In Stock')
axes[0, 1].hist(bootstrap_df['stockout_mean'].dropna(), bins=50, density=True, alpha=0.7, label='Stockout')
axes[0, 1].set_title('Bootstrap: Mean Demand by Stock Status')
axes[0, 1].set_xlabel('Mean Demand')
axes[0, 1].legend()

# Seasonal pattern with confidence bands
week_cols = [c for c in bootstrap_df.columns if c.startswith('week_')]
weeks = sorted([int(c.split('_')[1]) for c in week_cols])
weekly_means = [bootstrap_df[f'week_{w}'].mean() for w in weeks]
weekly_lower = [bootstrap_df[f'week_{w}'].quantile(0.025) for w in weeks]
weekly_upper = [bootstrap_df[f'week_{w}'].quantile(0.975) for w in weeks]

axes[1, 0].plot(weeks, weekly_means, 'b-', label='Mean', linewidth=2)
axes[1, 0].fill_between(weeks, weekly_lower, weekly_upper, alpha=0.3, label='95% CI')
axes[1, 0].set_title('Bootstrap: Seasonal Pattern (Retail Week)')
axes[1, 0].set_xlabel('Retail Week')
axes[1, 0].set_ylabel('Mean Demand')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Year-over-year with CI
year_cols = [c for c in bootstrap_df.columns if c.startswith('year_')]
years = sorted([int(c.split('_')[1]) for c in year_cols])
year_means = [bootstrap_df[f'year_{y}'].mean() for y in years]
year_lower = [bootstrap_df[f'year_{y}'].quantile(0.025) for y in years]
year_upper = [bootstrap_df[f'year_{y}'].quantile(0.975) for y in years]

axes[1, 1].errorbar(years, year_means, 
                     yerr=[np.array(year_means)-np.array(year_lower), 
                           np.array(year_upper)-np.array(year_means)],
                     fmt='o-', capsize=5, linewidth=2, markersize=8)
axes[1, 1].set_title('Bootstrap: Year-over-Year Trend')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Mean Demand')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/bootstrap_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"💾 Saved: ../reports/bootstrap_analysis.png")


## 8. Summary & Key Findings

All analyses complete! Artifacts saved to `data/processed/` and `reports/`.


In [ ]:
print("=" * 80)
print("COMPREHENSIVE TIME SERIES EDA - SUMMARY")
print("=" * 80)

print(f"\n📊 Dataset:")
print(f"  • {df.shape[0]:,} observations (SKU-weeks)")
print(f"  • {df['Store'].nunique()} stores × {df['Product'].nunique()} products")
print(f"  • {len(df['week'].unique())} weeks ({df['week_date'].min()} to {df['week_date'].max()})")

print(f"\n📈 Summary Statistics:")
print(f"  • Mean CV: {summary['cv'].mean():.2f} (high dispersion)")
print(f"  • Skewness (median): {summary['skewness'].median():.2f}")
print(f"  • Intermittent SKUs (ADI>1.32): {(summary['adi'] > 1.32).sum()} / {len(summary)} ({(summary['adi'] > 1.32).mean()*100:.1f}%)")
print(f"  • Stockout rate (median): {summary['pct_stockout'].median()*100:.1f}%")

print(f"\n🔬 Stationarity Tests:")
print(f"  • ADF stationary: {stationarity_tests['adf_stationary'].sum()} / {len(stationarity_tests)} ({stationarity_tests['adf_stationary'].mean()*100:.1f}%)")
print(f"  • KPSS stationary: {stationarity_tests['kpss_stationary'].sum()} / {len(stationarity_tests)} ({stationarity_tests['kpss_stationary'].mean()*100:.1f}%)")
print(f"  • Heteroskedastic: {stationarity_tests['heteroskedastic'].sum()} / {len(stationarity_tests)} ({stationarity_tests['heteroskedastic'].mean()*100:.1f}%)")

print(f"\n📡 Frequency Analysis:")
print(f"  • Median dominant period: {dominant_periods_df['top_period'].median():.1f} weeks")
print(f"  • SKUs with clear seasonality: {dominant_periods_df['n_dominant_periods'].sum()}")

print(f"\n🔄 Bootstrap Results (10,000 iterations):")
print(f"  • Overall mean demand: {bootstrap_df['overall_mean'].mean():.2f} ± {bootstrap_df['overall_mean'].std():.3f}")
print(f"  • In-stock vs stockout difference: {bootstrap_df['instock_mean'].mean() - bootstrap_df['stockout_mean'].mean():.2f}")

print(f"\n💾 Artifacts Saved:")
print(f"  • ../data/processed/master_demand_slurp.xml")
print(f"  • ../data/processed/demand_long.parquet")
print(f"  • ../data/processed/summary_statistics.parquet")
print(f"  • ../data/processed/stationarity_tests.parquet")
print(f"  • ../data/processed/bootstrap_distributions.pkl")
print(f"  • ../reports/dispersion_analysis.png")
print(f"  • ../reports/acf_pacf_analysis.png")
print(f"  • ../reports/frequency_analysis.png")
print(f"  • ../reports/bootstrap_analysis.png")

print(f"\n✅ Comprehensive EDA complete! Ready for forecasting and optimization.")
print("=" * 80)
